In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

### Load Data

In [2]:
# Load the data as numpy arrays
img_1 = pd.read_csv('./../data/feaSubEImg_1.csv', header=None).to_numpy()
img_2 = pd.read_csv('./../data/feaSubEImg_2.csv', header=None).to_numpy()
overt_1 = pd.read_csv('./../data/feaSubEOvert_1.csv', header=None).to_numpy()
overt_2 = pd.read_csv('./../data/feaSubEOvert_2.csv', header=None).to_numpy()

In [3]:
# Combine the features
img_X = np.hstack((img_1, img_2)).T
img_y = np.array([0] * img_1.shape[1] + [1] * img_2.shape[1])

overt_X = np.hstack((overt_1, overt_2)).T
overt_y = np.array([0] * overt_1.shape[1] + [1] * overt_2.shape[1])

### Two-level Cross-validation function

In [4]:
# Inner cross-validation
def inner_cv(X_train, y_train, C_values=[0.01, 1, 100, 10000], n_splits=5):
    """
    Perform inner 5-fold cross-validation to find the best regularization parameter C.

    Parameters:
        X_train (np.ndarray): Training data (samples x features)
        y_train (np.ndarray): Labels (0 or 1)
        C_values (list): List of C values to try
        n_splits (int): Number of folds for inner CV

    Returns:
        best_C (float): The value of C with the highest average validation accuracy
        val_accuracies (dict): Mapping of C → list of validation accuracies
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True)
    val_accuracies = {C: [] for C in C_values}

    for train_idx, val_idx in skf.split(X_train, y_train):
        X_inner_train, X_val = X_train[train_idx], X_train[val_idx]
        y_inner_train, y_val = y_train[train_idx], y_train[val_idx]

        for C in C_values:
            clf = SVC(kernel='linear', C=C)
            clf.fit(X_inner_train, y_inner_train)
            y_val_pred = clf.predict(X_val)
            acc = accuracy_score(y_val, y_val_pred)
            val_accuracies[C].append(acc)

    # Average accuracy for each C
    avg_acc = {C: np.mean(accs) for C, accs in val_accuracies.items()}
    best_C = max(avg_acc, key=avg_acc.get)

    return best_C, val_accuracies

In [5]:
def two_level_cross_validation(X, y, outer_splits=6, inner_splits=5, C_values=[0.01, 1, 100, 10000]):
    """
    Performs two-level cross-validation:
    - Outer loop evaluates generalization performance (6-fold)
    - Inner loop tunes regularization parameter C (5-fold)

    Parameters:
        X (np.ndarray): (N, 204) feature matrix (trials x features)
        y (np.ndarray): (N,) label vector (0s and 1s)
        outer_splits (int): Number of outer folds (default=6)
        inner_splits (int): Number of inner folds (default=5)
        C_values (list): Regularization parameters to search over

    Returns:
        results (dict): {
            'accuracies': list of 6 outer test accuracies,
            'best_Cs': best C for each outer fold,
            'y_true_all': all test set labels (concatenated),
            'y_pred_all': all predicted labels,
            'decision_scores_all': all decision function values
        }
    """
    # Splits data into 6 folds
    outer_skf = StratifiedKFold(n_splits=outer_splits, shuffle=True)

    accuracies = []
    best_Cs = []
    y_true_all = []
    y_pred_all = []
    decision_scores_all = []

    # Outer CV loop. Iterate over each fold
    for fold_idx, (train_idx, test_idx) in enumerate(outer_skf.split(X, y)):
        # Split data into outer train and test sets
        X_train_outer, X_test_outer = X[train_idx], X[test_idx]
        y_train_outer, y_test_outer = y[train_idx], y[test_idx]

        # Inner CV loop to select best C
        best_C, _ = inner_cv(X_train_outer, y_train_outer, C_values=C_values, n_splits=inner_splits)
        best_Cs.append(best_C)

        # Train final model on full outer train set using best C
        model = SVC(kernel='linear', C=best_C)
        model.fit(X_train_outer, y_train_outer)

        # Evaluate on outer test set
        y_pred = model.predict(X_test_outer)
        decision_scores = model.decision_function(X_test_outer)
        acc = accuracy_score(y_test_outer, y_pred)

        # Save results
        accuracies.append(acc)
        y_true_all.extend(y_test_outer)
        y_pred_all.extend(y_pred)
        decision_scores_all.extend(decision_scores)

        print(f"[Fold {fold_idx+1}] Accuracy: {acc:.2f}, Best C: {best_C}")

    results = {
        'accuracies': accuracies,
        'best_Cs': best_Cs,
        'y_true_all': np.array(y_true_all),
        'y_pred_all': np.array(y_pred_all),
        'decision_scores_all': np.array(decision_scores_all)
    }

    return results